In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# Feature extraction
from sklearn.feature_selection import VarianceThreshold

from boruta import BorutaPy
from sklearn.inspection import permutation_importance

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform

# Classification models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data_path = "../../data/"

In [3]:
df_radiomics = pd.read_excel(data_path + 'quantitative_radiomic_features.xls')
df_radiomics['Lesion Name'] = df_radiomics['Lesion Name'].str.replace(r'-1\.les|-S2-1\.les', '', regex=True)

df_clinical = pd.read_excel(data_path + "clinical_data.xls")
df_clinical.replace("[Not Available]", np.nan, inplace=True)
df_clinical.rename(columns={'bcr_patient_barcode': 'Lesion Name'}, inplace=True)
df_clinical = df_clinical[['Lesion Name', 'age_at_initial_pathologic_diagnosis', 'ajcc_neoplasm_disease_stage', "ajcc_tumor_stage_code", "number_of_lymphnodes_positive_by_he",
                          "breast_carcinoma_estrogen_receptor_status", "breast_carcinoma_progesterone_receptor_status"]]

df_assays = pd.read_excel(data_path + "multigenic_assays.xlsx")
df_assays = df_assays.drop(columns=df_assays.columns[1])
df_assays.rename(columns={'CLID': 'Lesion Name'}, inplace=True)
df_assays = df_assays[['Lesion Name', 'GHI_RS Score', 'Mammaprint Pcorr_NKI70_Good_Correlation_Nature.2002_PMID.11823860', 
                        'UNC_Proliferation_11_Mean_JCO.2009_PMID.19204204']]

target_class = pd.read_csv(data_path + 'target_class.csv')
target_class.rename(columns={'CLID': 'Lesion Name'}, inplace=True)

/tmp/ipykernel_921/636512698.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clinical.replace("[Not Available]", np.nan, inplace=True)
/home/jose/anaconda3/envs/tfg/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
df = pd.merge(df_radiomics, target_class, on='Lesion Name', how='inner')
df = pd.merge(df, df_clinical, on='Lesion Name', how='inner')
df = pd.merge(df, df_assays, on='Lesion Name', how='inner')
df = df.drop(columns=df.columns[0])
df

,Maximum enhancement (K1),Time to peak (K2),Uptake rate (K3),Washout rate (K4),Curve shape index (K5),E1 (K6),Signal Enhancement Ratio (SER) (K7),Maximum enhancement-variance (E1),Enhancement-Variance Time to Peak (E2),Enhancement-variance Increasing Rate (E3),...,Pam50.Call,age_at_initial_pathologic_diagnosis,ajcc_neoplasm_disease_stage,ajcc_tumor_stage_code,number_of_lymphnodes_positive_by_he,breast_carcinoma_estrogen_receptor_status,breast_carcinoma_progesterone_receptor_status,GHI_RS Score,Mammaprint Pcorr_NKI70_Good_Correlation_Nature.2002_PMID.11823860,UNC_Proliferation_11_Mean_JCO.2009_PMID.19204204
0,1.602573,145.433,0.011019,0.000809,-0.099998,1.550204,1.111109,0.125963,60.000,0.002099,...,LumA,29,Stage IA,T1c,0.0,Positive,Positive,100.000000,0.458,-0.232964
1,4.072152,144.752,0.028132,0.001510,-0.007311,3.973258,1.007365,4.054312,229.504,0.017666,...,LumA,41,Stage IA,T1c,0.0,Positive,Positive,48.574519,0.628,-0.673673
2,1.303264,168.383,0.007740,0.000667,-0.146959,1.273733,1.172277,0.534411,60.000,0.008907,...,LumA,61,Stage IIIC,T2,15.0,Positive,Positive,55.213839,0.567,-0.263108
3,0.758630,60.000,0.012644,0.000479,-0.150482,0.758630,1.177138,0.037956,298.418,0.000127,...,LumA,56,Stage IIA,T1c,0.0,Positive,Negative,72.668351,0.420,-0.398098
4,3.353556,60.000,0.055893,0.000920,-0.080066,3.353556,1.087034,2.330160,60.000,0.038836,...,LumB,40,Stage IIB,T2,2.0,Positive,Positive,100.000000,0.121,0.233268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.277688,60.000,0.021295,0.000611,-0.149392,1.277688,1.175629,0.271695,60.000,0.004528,...,LumA,66,Stage IIA,T2,0.0,Positive,Positive,100.000000,0.319,0.061330
72,1.495674,60.000,0.024928,0.000093,-0.017541,1.495674,1.017854,0.123041,60.000,0.002051,...,LumA,46,Stage IIA,T2,0.0,Positive,Positive,38.471147,0.469,-0.341741
73,2.182750,60.000,0.036379,0.001081,-0.125678,2.182750,1.143744,0.225339,313.733,0.000718,...,Basal,44,Stage IIA,T2,0.0,Negative,Negative,100.000000,-0.541,0.263237
74,1.354828,192.786,0.007028,0.000067,0.043301,1.290058,0.958496,0.108778,60.000,0.001813,...,LumA,61,Stage IIB,T2,1.0,Positive,Positive,54.557275,0.704,-0.345247


In [5]:
from sklearn.preprocessing import StandardScaler

radiomic_features = df.drop(columns=["Pam50.Call", 'age_at_initial_pathologic_diagnosis', 'ajcc_neoplasm_disease_stage', "ajcc_tumor_stage_code", "number_of_lymphnodes_positive_by_he",
                                    "breast_carcinoma_estrogen_receptor_status", "breast_carcinoma_progesterone_receptor_status"])

scaler = StandardScaler()
scaled_features = scaler.fit_transform(radiomic_features)

df_scaled = pd.DataFrame(scaled_features, columns=radiomic_features.columns)

df_scaled["Pam50.Call"] = df["Pam50.Call"].values
df_scaled["age_at_initial_pathologic_diagnosis"] = df["age_at_initial_pathologic_diagnosis"].values
df_scaled["ajcc_neoplasm_disease_stage"] = df["ajcc_neoplasm_disease_stage"].values
df_scaled["ajcc_tumor_stage_code"] = df["ajcc_tumor_stage_code"].values
df_scaled["number_of_lymphnodes_positive_by_he"] = df["number_of_lymphnodes_positive_by_he"].values 
df_scaled["breast_carcinoma_estrogen_receptor_status"] = df["breast_carcinoma_estrogen_receptor_status"].values
df_scaled["breast_carcinoma_progesterone_receptor_status"] = df["breast_carcinoma_progesterone_receptor_status"].values 

df = df_scaled
df_dummies = pd.get_dummies(df, columns=['ajcc_neoplasm_disease_stage', "ajcc_tumor_stage_code",
                                        "breast_carcinoma_estrogen_receptor_status", 
                                         "breast_carcinoma_progesterone_receptor_status"], drop_first=True)

new_dummy_columns = set(df_dummies.columns) - set(df.columns)

df = df_dummies

In [6]:
features = df.columns.difference(['Pam50.Call'])
target_col = 'Pam50.Call'

X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target_col],
    test_size=0.2,
    random_state=42,
    stratify=df[target_col]
)

In [7]:
under_sampler = RandomUnderSampler(sampling_strategy={"LumA": 30}, random_state=42)
X_train_under, y_train_under = under_sampler.fit_resample(X_train, y_train)

over_sampler = SMOTE(sampling_strategy="not majority", random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = over_sampler.fit_resample(X_train_under, y_train_under)

print("\nDistribución tras Under-Sampling + Over-Sampling en train:")
print(y_train_resampled.value_counts())


Distribución tras Under-Sampling + Over-Sampling en train:
Pam50.Call
Basal    30
Her2     30
LumA     30
LumB     30
Name: count, dtype: int64


In [8]:
df_resampled = pd.concat(
    [pd.DataFrame(X_train_resampled), pd.Series(y_train_resampled, name='Pam50.Call')],
    axis=1
)

In [9]:
selected_features = ['Margin Sharpness (M1)', 'Maximum enhancement-variance (E1)', 'Surface Area (S3)'] + list(new_dummy_columns) + ['GHI_RS Score', 
                                             'Mammaprint Pcorr_NKI70_Good_Correlation_Nature.2002_PMID.11823860', 
                                             'UNC_Proliferation_11_Mean_JCO.2009_PMID.19204204']

In [10]:
X_train_resampled_selected = X_train_resampled[selected_features]
X_test_selected = X_test[selected_features]

In [11]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, features, target_col, model, imprimir=False):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    if imprimir:
        print("Classification Report:")
        print(classification_report(y_test, y_pred))
    return macro_f1

In [12]:
rf_model = RandomForestClassifier(random_state=42)

macro_f1_score = train_and_evaluate_model(
    X_train_resampled_selected, X_test_selected, y_train_resampled, y_test,
    features=[col for col in df.columns if col != "Pam50.Call"],
    target_col="Pam50.Call",
    model=rf_model,
    imprimir=True
)

Classification Report:
              precision    recall  f1-score   support

       Basal       1.00      1.00      1.00         2
        Her2       0.33      1.00      0.50         1
        LumA       1.00      0.82      0.90        11
        LumB       0.00      0.00      0.00         2

    accuracy                           0.75        16
   macro avg       0.58      0.70      0.60        16
weighted avg       0.83      0.75      0.78        16

